## iGraph tutorial

This notebook is for using the python bindings for iGraph, if you prefer R (ugh!) or even C/C++ (OMG!) refer to the corresponding documentation.

If you want to install python-igraph on windows or mac you should follow the documentation instructions http://igraph.org/python/#startpy

To generate scalable vector graphs (SVG) with the plot methods you need *pycairo* (it is not instalable using pip, you have to download it from http://cairographics.org/pycairo/) and the library *cairo* has to be installed in your linux). We will avoid using pycairo generating png files instead.

In this notebook it is also used `numpy` and `matplotlib`

Apart from this notebook, the tutorial for iGraph is a good starting point http://igraph.org/python/doc/tutorial/tutorial.html

In [ ]:
import sys
print(sys.version)
print(sys.version_info)

In [ ]:
#!pip3 install igraph --user
#!pip3 install pycairo --user
#!pip3 install python-igraph --user
#!pip3 install matplotlib

In [ ]:
import igraph
from igraph import Graph

---
## 1. Creating and manipulating graphs

Creating a graph is simple, this is an **empty** graph (print returns a representation of the graph)

In [ ]:
g = Graph()
print(g)

We can add vertices like for example, vertices from [0..3]

In [ ]:
g.add_vertices(4)
print(g)

and some edges among them (as a list of pairs)

In [ ]:
g.add_edges([(1,2), (1,3), (2,3), (3, 0)])
print(g)

Class `Graph` provides a couple of iterators to traverse the vertices (**vs**) and the edges (**es**)

In [ ]:
for v in g.vs:
    print(v)

In [ ]:
for e in g.es:
    print(e)

Properties can be attached to the vertices (and the edges)

In [ ]:
for i, v in enumerate(g.vs):
    v['name'] = str(i)
for v in g.vs:
    print(v)
    print(v.attributes())

In [ ]:
import random
for edge in g.es:
    edge['weight'] = random.randint(1,10)
for edge in g.es:
    print(edge)
    print(edge['weight'])

Class `Graph` has methods to compute properties of a graph and many useful algorithms (https://igraph.org/python/doc/api/igraph.Graph.html)


In [ ]:
g.degree()

In [ ]:
g.edge_betweenness()

---
## 2. Plotting graphs

Plotting the graph is a little bit more complicated because by default it uses the Cairo library.

If the cairo library is not installed on your machine, we can circunvent this problem by generating a `png` file and loading it in the notebook. There are different styles and layouts for plotting a graph, you can look for the parameters in the documentation of the plot function in the Graph class.

If the cairo library is installed on your computer, then saving the `.png` file and displaying it is not necessary.

In [ ]:
from IPython import display
from igraph import plot

# with cairo:

g.vs['label'] = g.vs['name']
g.es['width'] = g.es['weight']
plot(g, layout = g.layout_circle(), bbox = (300,300))

---
## 3. Graph Generators

iGraph implements several graph generators (Erdos-Renyi, Barabasi, Watts-Strogratz, ...). This should be very useful to solve your lab tasks for today.

Different layouts can be used for plotting the graphs; these layouts define where vertices are places (in 2D) in order to visualize your generated networks.

In [ ]:
erdos = Graph.Erdos_Renyi(50,0.05)
plot(erdos)

In [ ]:
barabasi = Graph.Barabasi(100,1)
plot(barabasi, layout = barabasi.layout_fruchterman_reingold())

In [ ]:
watts = Graph.Watts_Strogatz(1,100,2,0.05)
plot(watts, layout = watts.layout_lgl())

---
## 4. Measuring graphs

There are many measures that help us understand and characterize networks. We have seen three in class: _diameter_ (or _average path length_), _clustering coefficient_ (or _transitivity_), and _degree distribution_. `igraph` provides functions that compute these measures for you. 

The examples below illustrate the usage of these functions.

#### For a lattice network

In [ ]:
lattice = Graph.Lattice([10,10])
plot(lattice, layout=lattice.layout_kamada_kawai())

In [ ]:
lattice.diameter(), lattice.average_path_length()

In [ ]:
# here we rewire 20% of the edges randomly, and see how it affects the average path length and diameter (should shrink)

lattice.rewire_edges(0.2)
lattice.diameter(), lattice.average_path_length()

#### For a Watts-Strogatz graph

In [ ]:
p_hat = len(watts.es)/(len(watts.vs)*len(watts.vs)/2)
p_hat, watts.transitivity_undirected()

#### For a ER graph

In [ ]:
p_hat = len(erdos.es)/(len(erdos.vs)*len(erdos.vs)/2)
p_hat, erdos.transitivity_undirected()

#### For a Ring graph

In [ ]:
ring = Graph.Ring(10)
plot(ring, layout=ring.layout_kamada_kawai(), bbox=(150,150))

In [ ]:
p_hat = len(ring.es)/(len(ring.vs)*len(ring.vs)/2)
p_hat, ring.transitivity_undirected()

In [ ]:
ring.degree()

#### For a BA graph

In [ ]:
barabasi = Graph.Barabasi(1000,3)
p_hat = len(barabasi.es)/((len(barabasi.vs)-1)*len(barabasi.vs)/2.0)
p_hat, barabasi.transitivity_undirected()

In [ ]:
# generate ER graph with same nr of vertices and approximately same number of edges as BA network
erdos = Graph.Erdos_Renyi(1000, p_hat)

In [ ]:
import numpy as np
h, b = np.histogram(erdos.degree(), bins=np.max(erdos.degree()), density=True)
print(h)
print(b)

In [ ]:
import matplotlib.pyplot as plt

r = plt.hist(erdos.degree(), bins=np.max(erdos.degree()))

In [ ]:
r = plt.plot(b[1:], h, 'o' )

In [ ]:
r = plt.hist(barabasi.degree(), bins=np.max(barabasi.degree()))

In [ ]:
h, b = np.histogram(barabasi.degree(), bins=np.max(barabasi.degree()), density=True)
r = plt.plot(b[1:], h, 'o')

In [ ]:
r = plt.loglog(b[1:], h, 'o')

---
## 5. Node centrality

First we creeate a graph following the Erdos-Reny model

In [ ]:
erdos = Graph.Erdos_Renyi(20,0.2)

plot(erdos, layout = erdos.layout_kamada_kawai(), bbox=(400,300))

#### **Betweenness** centrality

In [ ]:
erdos.betweenness()

#### **Degree** centrality

In [ ]:
erdos.degree()

#### **Closeness** centrality

In [ ]:
erdos.closeness()

####  **Pagerank** centrality

In [ ]:
erdos.pagerank()

---
## 6. Community detection


In [ ]:
com = erdos.community_edge_betweenness()
print ('Clusters:', com.optimal_count) # Optimal number of clusters from the dendrogram

the `as_clustering` method of the dendrogram "cuts" the dendogramobject returns a graph with the clusters:

In [ ]:
r = plot(com.as_clustering(), layout = erdos.layout_kamada_kawai(),mark_groups=True, target="./coms.png")
display.Image(filename="./coms.png")

You can also obtain the clusters and their vertices

In [ ]:
for d in com.as_clustering():
    print(d)

In [ ]:
print(com.as_clustering())